[![LibKet](../images/LibKet.png)](https://gitlab.com/mmoelle1/LibKet)
**LibKet - The Quantum Expression Template Library.**
- Repository:    https://gitlab.com/mmoelle1/LibKet/
- Documentation: https://libket.readthedocs.io/
- API docs:      https://mmoelle1.gitlab.io/LibKet/

***

# Tutorial \#2: Hands-on Scientific Computing with LibKet - Part 1

>In this tutorial you will learn to
> 1. offload quantum expressions to different quantum devices
> 2. customize the offloading pipeline
> 3. visualize results using Qiskit

## Getting started

Let's include **LibKet**'s main headerfile and import its namespaces. This can take some time, stay tuned.

In [ ]:
#include "LibKet.hpp"
using namespace LibKet;
using namespace LibKet::circuits;
using namespace LibKet::filters;
using namespace LibKet::gates;

Let us create a simple quantum expression for the first Bell state

![Bell state](../images/multi_qubit_circuit_HI_CNOT.png)

and evaluate it on different quantum backends. Don't forget to ``measure`` at the end since some backends do not support direct statevector readouts.

In [ ]:
auto expr = measure(cnot(h(sel<0>()), sel<1>(init())));

## Cirq

![Cirq](../images/cirq_logo.png)

[Cirq](https://quantumai.google/cirq) is a Python package for writing, manipulating, and optimizing quantum circuits developed by Google. It is seamlessly integrated into **LibKet** using the embedded Python interpreter, so you should not notice all the technical details going on under the hood.

As for the [QuEST](https://quest.qtechtheory.org) simulator, we simply create a quantum device for two qubits, load the quantum expression into it, and evaluate it.

In [ ]:
QDevice<QDeviceType::cirq_simulator, 2> cirq;
utils::json result = cirq(expr).eval(1);

Unlike QuEST, Cirq returns the outcome of the evaluation as JSON object which we can print directly

In [ ]:
std::cout << result << std::endl;

We can also get a slightly prettier output

In [ ]:
std::cout << result.dump(2) << std::endl;

As this is still not too informative, **LibKet** provides quick access `get<...>()` functions to retrieve information about the ``duration`` of the quantum computation, a ``histogram`` of all measured states, and the ``best`` state, i.e. the one that was measured most often

In [ ]:
std::cout << "duration  : " << cirq.get<QResultType::duration>(result).count() << " seconds" << std::endl;
std::cout << "histogram : " << cirq.get<QResultType::histogram>(result)        << std::endl;
std::cout << "best      : " << cirq.get<QResultType::best>(result)             << std::endl;

Let's increase the number of **shots**, i.e. the number of times we run the quantum circuit in our simulator, to ``20`` to see how the histogram changes 

In [ ]:
result = cirq.eval(20);

std::cout << "duration  : " << cirq.get<QResultType::duration>(result).count() << " seconds" << std::endl;
std::cout << "histogram : " << cirq.get<QResultType::histogram>(result)        << std::endl;
std::cout << "best      : " << cirq.get<QResultType::best>(result)             << std::endl;

## Qiskit

![Qiskit](../images/qiskit_logo.png)

Let's move on to the next quantum backend supported by **LibKet** - [Qiskit](https://qiskit.org). Without any changes to the quantum expression ``expr``, we create a new Qiskit device, populate it with the quantum expression and run it the same way as before

In [ ]:
QDevice<QDeviceType::qiskit_qasm_simulator, 2> qiskit;
utils::json result = qiskit(expr).eval(20);

std::cout << "duration  : " << qiskit.get<QResultType::duration>(result).count() << " seconds" << std::endl;
std::cout << "histogram : " << qiskit.get<QResultType::histogram>(result)        << std::endl;
std::cout << "best      : " << qiskit.get<QResultType::best>(result)             << std::endl;

## Rigetti

![Rigetti](../images/rigetti_logo.png)

Our third backend, [Rigetti](https://www.rigetti.com), requires some additional configuration as it relies on a quantum virtual machine ([qvm](https://github.com/rigetti/qvm)) and a quantum compiler toolchain ([quilc](https://github.com/rigetti/quilc)). Both tools can be [installed](https://pyquil-docs.rigetti.com/en/stable/start.html) locally or on remote servers. You will be given the up-to-date IP addresses of our servers that run pre-built [docker images](https://hub.docker.com/u/rigetti) during the tutorial session.

The easiest way to customize quantum devices is by means of the JSON-based constructor

In [ ]:
utils::json config = { 
    { "qvm_url",   "http://68.183.13.151:80"  },
    { "quilc_url", "tcp://104.248.201.181:80" }
};

QDevice<QDeviceType::rigetti_9q_square_simulator, 2> rigetti(config);
utils::json result = rigetti(expr).eval(20);

std::cout << "duration  : " << rigetti.get<QResultType::duration>(result).count() << " seconds" << std::endl;
std::cout << "histogram : " << rigetti.get<QResultType::histogram>(result)        << std::endl;
std::cout << "best      : " << rigetti.get<QResultType::best>(result)             << std::endl;

Let's take a short look under the hood to see how much work is required to write low-level quantum assembly language (QASM) for the three simulators

In [ ]:
std::cout << "---- Cirq ----\n" << cirq    << std::endl;
std::cout << "--- Qiskit ---\n" << qiskit  << std::endl;
std::cout << "-- Rigetti ---\n" << rigetti << std::endl;

**LibKet** is designed as backend-agnostic quantum programming framework with a unified API, which means that the core functionality is implemented for all quantum backends. If a particular backend does not support a specific function, e.g., Cirq does not report the ``duration`` of the quantum computation, default values are returned.

Some quantum backends provide extra functionality, which is exposed via **LibKet**. Cirq for instance can export the quantum circuit to [LaTeX](https://www.latex-project.org) code using the [Qcircuit](https://ctan.org/pkg/qcircuit) package

In [ ]:
std::cout << cirq.to_latex() << std::endl;

In [ ]:
std::cout << qiskit.to_latex() << std::endl;

The resulting circuit looks as follows

![Bell state](../images/qiskit_to_latex.png)

## Computation offloading

**LibKet**'s computation offloading model is very similar to that of [CUDA](https://developer.nvidia.com/cuda-downloads) to ease the transition from GPU- to quantum-accelerated computing. The `device.eval(...)` is just one of three ways to run a quantum expression, which we will refer to as **quantum kernel**, on a quantum device.

#### ``LibKet::utils::json device.eval(...)`` 
offloads the quantum computation to the quantum device and returns the evaluated result as JSON object once the quantum computation has completed. Exceptions are the QuEST and QX simulators where a reference to the internal state vector is returned.

#### ``LibKet::QJob* device.execute(...)`` 
offloads the quantum computation to the quantum device and returns a ``QJob`` pointer once the quantum computation has completed.

#### ``LibKet::QJob* device.execute_async(...)`` 
offloads the quantum computation to the quantum device and returns a ``QJob`` pointer immediately.

All three methods have the same interface. For Python-based quantum backends this is
```cpp
QJob<QJobType::Python>* execute(std::size_t shots                 = [default from ctor],
                                const std::string& script_init    = "",
                                const std::string& script_before  = "",
                                const std::string& script_after   = "",
                                QStream<QJobType::Python>* stream = NULL)
```
and for backends implemented in C/C++ like QuEST and QX this is
```cpp
QJob<QJobType::CXX>*    execute(std::size_t                         shots       = [default from ctor],
                                std::function<void(QDevice_QuEST*)> ftor_init   = NULL,
                                std::function<void(QDevice_QuEST*)> ftor_before = NULL,
                                std::function<void(QDevice_QuEST*)> ftor_after  = NULL,
                                QStream<QJobType::CXX>*             stream      = NULL)
```

Let's change ``qiskit.eval(20)`` into ``qiskit.execute_async(20)`` to offload the execution of the quantum kernel to the ``qiskit`` device. Since this cloud-based environment does not allow non-blocking execution this and ``qiskit.execute(20)`` behave identically. If used in a regular program, asynchronous execution immediately returns the scope to the host program and passes a pointer to the ``QJob`` object.

In [ ]:
auto job = qiskit.execute_async(20);

``QJob`` objects support the following functionality

#### ``QObj* wait()`` 
waits for the job to complete (blocking)
#### ``bool query()`` 
returns ``true`` if the job completed and ``false`` otherwise (non-blocking)
#### ``utils::json get()`` 
returns the result as JSON object after completion (blocking)

Let's collect the result of our quantum kernel execution

In [ ]:
result = job->get();
std::cout << result.dump(2) << std::endl;

## Customizing quantum kernel execution

The optional hooks ``script_init``, ``script_before``, and ``script_after`` and their C++ counterparts ``ftor_init``, ``ftor_before``, and ``ftor_after`` make it possible to inject user-defined code at three different locations of the execution process:

#### ``script_init``
is performed before any other code of the execution process. It can be used for importing additional Python modules
#### ``script_before``
is performed just before sending the instructions to the quantum device. It can be used to pre-process the quantum circuit, e.g., to perform user-specific optimizations on the raw quantum circuit, before it runs through the backend-specific pipeline
#### ``script_after``
is performed just after receiving the result from the quantum device. It can be used to post-process the raw results received from the quantum device, e.g., to generate histograms or other types of visualizations

Let's inject a simple statement after the execution that collects the histogram data of the experiment using Qiskit's [``get_count()``](https://qiskit.org/documentation/stubs/qiskit.result.Result.html#qiskit.result.Result.get_counts) function

In [ ]:
auto job = qiskit.execute_async(20,
                                /* init_script   */
                                "",
                                /* before_script */
                                "",
                                /* after_script */
                                "counts = result.get_counts(qc)\n"
                                "return json.dumps(counts)\n"
                               );
std::cout << job->get().dump(2) << std::endl;

It should be noted that the code injections are idented automatically and must not have trailing ``\t``'s. Each line must end with ``\n``.

Qiskit comes with a set of [visualizations](https://qiskit.org/documentation/apidoc/visualization.html). Let's inject code that generates a histogram plot and saves it to a file named ``'histogram.png'``.

In [ ]:
auto job = qiskit.execute_async(20,
                                /* init_script   */
                                "import base64\n"
                                "from qiskit.visualization import plot_histogram\n",
                                /* before_script */
                                "",
                                /* after_script */
                                "counts = result.get_counts(qc)\n"
                                "plot_histogram(counts, color='midnightblue', title=\"Histogram\").savefig('histogram.png')\n"
                               );

On a regular computer you could simply open the file ``'histogram.png'``. [Xeus-cling](https://xeus-cling.readthedocs.io/en/latest/rich_display.html), which powers this cloud-based tutorial, has the ability to display rich content. All it needs is the following code snippet, which is already included in this environment

```cpp
#include <string>
#include <fstream>
#include "xtl/xbase64.hpp"

namespace image
{
    struct png
    {
        inline png(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);
            m_buffer << fin.rdbuf();
        }

        std::stringstream m_buffer;
    };

    utils::json mime_bundle_repr(const png& i)
    {
        auto bundle = utils::json::object();
        bundle["image/png"] = xtl::base64encode(i.m_buffer.str());
        return bundle;
    }
}
```

Let's have a look at our histogram

In [ ]:
image::png histogram("histogram.png");
histogram

Try yourself some of Qiskit's other visualizations. You can use ``image::png`` directly without copying the above code snippet.

* [qiskit.visualization.plot_bloch_multivector](https://qiskit.org/documentation/stubs/qiskit.visualization.plot_bloch_multivector.html#qiskit.visualization.plot_bloch_multivector)
* [qiskit.visualization.plot_state_city](https://qiskit.org/documentation/stubs/qiskit.visualization.plot_state_city.html#qiskit.visualization.plot_state_city)
* [qiskit.visualization.plot_state_hinton](https://qiskit.org/documentation/stubs/qiskit.visualization.plot_state_hinton.html#qiskit.visualization.plot_state_hinton)
* [qiskit.visualization.plot_state_paulivec](https://qiskit.org/documentation/stubs/qiskit.visualization.plot_state_paulivec.html#qiskit.visualization.plot_state_paulivec)
* [qiskit.visualization.plot_state_qsphere](https://qiskit.org/documentation/stubs/qiskit.visualization.plot_state_qsphere.html#qiskit.visualization.plot_state_qsphere)